In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import *

In [2]:
df = pd.read_csv("/content/drive/MyDrive/Datasets/creditcard.csv")
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
df['Class'].value_counts(normalize = True)

,proportion
Class,
0,0.998273
1,0.001727


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [6]:
df.isnull().sum()

,0
Time,0
V1,0
V2,0
V3,0
V4,0
V5,0
V6,0
V7,0
V8,0
V9,0


In [7]:
df.duplicated().sum()

np.int64(1081)

In [8]:
df.drop_duplicates(inplace = True)

In [9]:
col = df[['Time','Amount']]

scaler = StandardScaler()

df[['Time','Amount']] = scaler.fit_transform(col)

In [10]:
X = df.drop(columns = 'Class')
y = df['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, stratify = y, random_state = 42)


In [11]:
smote = SMOTE(random_state = 42)

X_train_re, y_train_re = smote.fit_resample(X_train, y_train)

In [12]:
log_model = LogisticRegression()
log_model.fit(X_train_re, y_train_re)
y_pred = log_model.predict(X_test)
y_pred_proba = log_model.predict_proba(X_test)[:,1]

In [14]:
log_precision = precision_score(y_test,y_pred)
log_recall = recall_score(y_test,y_pred)
log_f1 =f1_score(y_test,y_pred)
log_roc_auc = roc_auc_score(y_test,y_pred_proba)

print(f"Precision: {log_precision:.2f}, Recall: {log_recall:.2f}, F1-Score {log_f1:.2f}, Roc_auc: {log_roc_auc:.2f}")

Precision: 0.05, Recall: 0.87, F1-Score 0.10, Roc_auc: 0.96


In [15]:
params = {
    'n_estimators' : [50,100,150],
    'max_depth' : [2,4],
    'min_samples_split' : [2,5,6],
    'min_samples_leaf' :[2,3,4],
    'criterion':['gini', 'entropy']
}

model = RandomForestClassifier()
cv = RandomizedSearchCV(estimator = model, param_distributions = params, n_iter = 10)
cv.fit(X_train_re, y_train_re)
cv.best_params_

{'n_estimators': 100,
 'min_samples_split': 6,
 'min_samples_leaf': 4,
 'max_depth': 4,
 'criterion': 'gini'}

In [18]:
random_model = RandomForestClassifier(n_estimators = 100 , max_depth = 4, min_samples_split = 6, min_samples_leaf =4 , criterion ='gini')
random_model.fit(X_train_re, y_train_re)
y_pred_r = random_model.predict(X_test)
y_pred_proba_r = random_model.predict_proba(X_test)[:,1]

In [19]:
r_precision = precision_score(y_test,y_pred_r)
r_recall = recall_score(y_test,y_pred_r)
r_f1 =f1_score(y_test,y_pred)
r_roc_auc = roc_auc_score(y_test,y_pred_proba_r)

print(f"Precision: {r_precision:.2f}, Recall: {r_recall:.2f}, F1-Score {r_f1:.2f}, Roc_auc: {r_roc_auc:.2f}")

Precision: 0.22, Recall: 0.82, F1-Score 0.10, Roc_auc: 0.97


In [21]:
xgb = XGBClassifier()
xgb.fit(X_train_re, y_train_re)
y_pred_Xgb = xgb.predict(X_test)
y_pred_proba_xgb = xgb.predict_proba(X_test)[:,1]

In [22]:
xgb_precision = precision_score(y_test,y_pred_Xgb)
xgb_recall = recall_score(y_test,y_pred_Xgb)
xgb_f1 = f1_score(y_test, y_pred_Xgb)
xgb_roc_auc = roc_auc_score(y_test,y_pred_proba_xgb)

print(f"Precision: {xgb_precision:.2f}, Recall: {xgb_recall:.2f}, f1-score : {xgb_f1:.2f}, Roc_auc: {xgb_roc_auc:.2f}")

Precision: 0.75, Recall: 0.79, f1-score : 0.77, Roc_auc: 0.97


In [27]:
threshold = 0.3
new_y_pred = (y_pred_proba_r >= threshold).astype(int)

In [28]:
fraud_df = pd.DataFrame(
    {
        'fraud_probability':y_pred_proba_r,
        'Predicted_class': new_y_pred,
        'Actual_class': y_test
    }
)
fraud_df.head()

,fraud_probability,Predicted_class,Actual_class
86568,0.066035,0,0
251557,0.091550,0,0
20232,0.251647,0,0
68952,0.093347,0,0
191852,0.212378,0,0


In [29]:
fraud_df['Amount'] = X_test['Amount'].values

fraud_df.sort_values(by = 'fraud_probability', ascending = False).head(15)

,fraud_probability,Predicted_class,Actual_class,Amount
119781,0.998116,1,1,0.143999
150601,0.998116,1,1,-0.349333
15476,0.998110,1,1,0.045996
12156,0.998110,1,0,0.006060
15736,0.998110,1,1,0.045996
120085,0.998110,1,0,0.085573
43281,0.998110,1,0,-0.349333
150654,0.998110,1,1,-0.353287
150644,0.998110,1,1,-0.050370
12468,0.998110,1,0,0.006060
